In [17]:
# !pip install feather-format
import pyarrow.feather as feather
titanic_train = feather.read_feather('titanic_train.feather')
titanic_train.head()

,grupo,id,HomePlanet,CryoSleep,deck,room_num,side,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,fname,lname,Transported,gastos
0,1.0,1.0,Europa,False,B,0.0,P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham,Ofracculy,False,0.0
1,2.0,1.0,Earth,False,F,0.0,S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna,Vines,True,736.0
2,3.0,1.0,Europa,False,A,0.0,S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark,Susent,False,10383.0
3,3.0,2.0,Europa,False,A,0.0,S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam,Susent,False,5176.0
4,4.0,1.0,Earth,False,F,1.0,S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy,Santantines,True,1091.0


In [6]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

aplicaremos los siguientes modelos
* TSNE
* K-means
* Linear Regression
* Logistic Regression 
* Random Forest
* Boosted Forest
* grid search
* PCA
* KNN classifier
* support vector (not in sklearn)

Manual de supervivencia
1. Escoje un modelo
1. Escoje los hiper-parámetros del modelo
1. Prepara el dataset en features matrix y el vector target
1. Divide el dataset en train y test
1. Aplica el algoritmo a tus datos de entrenamiento
1. Aplica el algoritmo a tus datos de prueba
1. Evalúa el modelo
1. Aplica el modelo en nuevos datos

In [7]:
# preparamos nuestro dataset en featura matrix y vector target



NameError: name 'titanic_train' is not defined

In [ ]:
modelo = LinearRegression(fit_intercept = True)
## 3.Prepara el /dataset/ en /features matrix/ y el vector /target/
## x no es una matriz, es un vector, i.e. su shape es (50,)
X = titanic_train.iloc[:,0][:, np.newaxis]
print(X.shape)

In [49]:
## 4. Divide el dataset en train y test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
## 5. Aplica el algoritmo a tu datos de entrenamiento
modelo.fit(X_train, y_train)

print("Coeficientes: {}".format(modelo.coef_))

print("Interceptor: {}".format(modelo.intercept_))

NameError: name 'y' is not defined

In [50]:
## 6. Aplica el algoritmo a tus datos de prueba
y_from_model = modelo.predict(X_test)

## 7. Evalúa el modelo
modelo.score(X_test, y_test) ## Devuelve el R^2x

NameError: name 'X_test' is not defined

In [51]:
## 8. Aplica el modelo en nuevos datos
x_new = np.linspace(-30, 30)

X_new = x_new[:, np.newaxis]
y_new = modelo.predict(X_new)
## Graficamos el modelo y los datos
plt.scatter(x, y)
plt.plot(X_new, y_new, c='red')


NotFittedError: This LinearRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [1]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=1000)
model.fit(X_train, y_train)
y_model = model.predict(X_test)
## Accuracy
from sklearn.metrics import accuracy_score
print("Accuracy: {}".format(accuracy_score(y_test, y_model)))
## Métricas clasicas de Clasificadores
from sklearn import metrics
print(metrics.classification_report(y_model, y_test))

NameError: name 'X_train' is not defined

In [2]:
classifier_names = ['LR',
                    'kNN',
                    'RBF SVM',
                    'Random Forest',
                    'Boosted Trees']
classifiers = [LogisticRegression(random_state=seed,solver='lbfgs'),
               KNeighborsClassifier(5),
               SVC(gamma=2, C=1, random_state=seed),
               RandomForestClassifier(max_depth=5, n_estimators=10, 
                                      max_features=1, random_state=seed),
               GradientBoostingClassifier(n_estimators=10, learning_rate=1.0,
                                          max_depth=5, random_state=seed)]
for model in classifiers:
    model.fit(training_data, training_labels)

NameError: name 'LogisticRegression' is not defined

In [3]:
def test_roc(model, data, labels):
    if hasattr(model, "decision_function"):
        predictions = model.decision_function(data)
    else:
        predictions = model.predict_proba(data)[:,1]
    fpr, tpr, thr = sklearn.metrics.roc_curve(labels, predictions)
    return fpr, tpr, thr

def test_conf(model, data, labels, thr):
    if hasattr(model, "decision_function"):
        predictions = model.decision_function(data)
    else:
        predictions = model.predict_proba(data)[:,1]
    mat = confusion_matrix(labels, predictions>=thr)
    return mat

In [5]:
for name, model in zip(classifier_names, classifiers):
    fpr, tpr, thr = test_roc(model, test_data, test_labels)
    plt.plot(fpr, tpr, label=name)
    print(name+' AUC:{}'.format(sklearn.metrics.auc(fpr, tpr)))
    J = tpr - fpr
    ix = argmax(J)
    best_thresh = thr[ix]
    print(name+' Best Threshold=%f' % (best_thresh))
    mat = test_conf(model, test_data, test_labels, best_thresh)
    print(mat)
    print()

NameError: name 'classifiers' is not defined